In [1]:
# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix, accuracy_score
import sklearn.metrics as metrics

### Load the Dataset 

In [5]:
df = pd.read_csv("Weather_Data.csv")
df.head(10)

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2/1/2008,19.5,22.4,15.6,6.2,0.0,W,41,S,SSW,...,92,84,1017.6,1017.4,8,8,20.7,20.9,Yes,Yes
1,2/2/2008,19.5,25.6,6.0,3.4,2.7,W,41,W,E,...,83,73,1017.9,1016.4,7,7,22.4,24.8,Yes,Yes
2,2/3/2008,21.6,24.5,6.6,2.4,0.1,W,41,ESE,ESE,...,88,86,1016.7,1015.6,7,8,23.5,23.0,Yes,Yes
3,2/4/2008,20.2,22.8,18.8,2.2,0.0,W,41,NNE,E,...,83,90,1014.2,1011.8,8,8,21.4,20.9,Yes,Yes
4,2/5/2008,19.7,25.7,77.4,4.8,0.0,W,41,NNE,W,...,88,74,1008.3,1004.8,8,8,22.5,25.5,Yes,Yes
5,2/6/2008,20.2,27.2,1.6,2.6,8.6,W,41,W,ENE,...,69,62,1002.7,998.6,6,6,23.8,26.0,Yes,Yes
6,2/7/2008,18.6,26.3,6.2,5.2,5.2,W,41,W,S,...,75,80,999.0,1000.3,4,7,21.7,22.3,Yes,Yes
7,2/8/2008,17.2,22.3,27.6,5.8,2.1,W,41,S,SE,...,77,61,1008.3,1007.4,7,8,18.9,21.1,Yes,Yes
8,2/9/2008,16.4,20.8,12.6,4.8,3.0,W,41,SSW,W,...,92,91,1006.4,1007.6,7,7,17.1,16.5,Yes,Yes
9,2/10/2008,14.6,24.2,8.8,4.4,10.1,W,41,W,SSE,...,80,53,1014.0,1013.4,4,2,17.2,23.3,Yes,No


### Data Preprocessing

In [6]:
# Perform one hot encoding to convert categorical vaiables to binary variables
df_sydney_processed = pd.get_dummies(data=df, columns=['RainToday', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

# Replace the values binary-converted variables into interger value of 0 or 1
df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)

# Drop the Date variable which is not used, and convert all variables value into float datatype
df_sydney_processed.drop('Date',axis=1,inplace=True)
df_sydney_processed = df_sydney_processed.astype(float)

df_sydney_processed.head(10)

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,19.5,22.4,15.6,6.2,0.0,41.0,17.0,20.0,92.0,84.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,19.5,25.6,6.0,3.4,2.7,41.0,9.0,13.0,83.0,73.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21.6,24.5,6.6,2.4,0.1,41.0,17.0,2.0,88.0,86.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20.2,22.8,18.8,2.2,0.0,41.0,22.0,20.0,83.0,90.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19.7,25.7,77.4,4.8,0.0,41.0,11.0,6.0,88.0,74.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,20.2,27.2,1.6,2.6,8.6,41.0,9.0,22.0,69.0,62.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,18.6,26.3,6.2,5.2,5.2,41.0,15.0,15.0,75.0,80.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,17.2,22.3,27.6,5.8,2.1,41.0,7.0,15.0,77.0,61.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,16.4,20.8,12.6,4.8,3.0,41.0,19.0,9.0,92.0,91.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,14.6,24.2,8.8,4.4,10.1,41.0,11.0,20.0,80.0,53.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Divide dataframe into features as independent variables and Y as a dependent variable(target)
features = df_sydney_processed.drop(columns='RainTomorrow', axis=1)
Y = df_sydney_processed['RainTomorrow']

# Split data into training and test data with scale of 8:2, respectively for training and testing 
x_train, x_test, y_train, y_test = train_test_split(features, Y, test_size=0.2, random_state=10)

In [8]:
# Initialize various machine learning models 
LinearReg = LinearRegression()
KNN = KNeighborsClassifier(n_neighbors=4)
Tree = DecisionTreeClassifier(max_depth=4, random_state=35)
LR = LogisticRegression(solver='liblinear')
SVM = svm.SVC(kernel='poly')

### Linear Regression

In [10]:
#Train the Linear Regression Model 
LinearReg.fit(x_train, y_train)

# Predict trained model using data test
LinearReg_predictions = LinearReg.predict(x_test)

In [11]:
#Calculate the Evaluation Metrics 
LinearRegression_MAE = metrics.mean_absolute_error(y_test, LinearReg_predictions)
LinearRegression_MSE = metrics.mean_squared_error(y_test, LinearReg_predictions)
LinearRegression_R2 = metrics.r2_score(y_test, LinearReg_predictions)

### KNN 

In [14]:
#Train the K Nearest Neighbors Model 
KNN.fit(x_train, y_train)

#Predict trained model using data test
KNN_predictions = KNN.predict(x_test)

In [15]:
#Calculate the Evaluation Metrics 
KNN_Accuracy_Score = metrics.accuracy_score(y_test, KNN_predictions)
KNN_JaccardIndex = metrics.jaccard_score(y_test, KNN_predictions)
KNN_F1_Score = metrics.f1_score(y_test, KNN_predictions)

### Decision Trees

In [16]:
#Train the Decision Trees Model 
Tree.fit(x_train, y_train)

#Predict trained model using data test
Tree_predictions = Tree.predict(x_test)

In [17]:
#Calculate the Evaluation Metrics 
Tree_Accuracy_Score = metrics.accuracy_score(y_test, Tree_predictions)
Tree_JaccardIndex = metrics.jaccard_score(y_test, Tree_predictions)
Tree_F1_Score = metrics.f1_score(y_test, Tree_predictions)

### Logistic Regression

In [19]:
#Train the Logistic Regression Model 
LR.fit(x_train, y_train)

#Predict trained model using data test
LR_predictions = LR.predict(x_test)
LR_predict_proba = LR.predict_proba(x_test)

In [20]:
#Calculate the Evaluation Metrics 
LR_Accuracy_Score = metrics.accuracy_score(y_test, LR_predictions)
LR_JaccardIndex = metrics.jaccard_score(y_test, LR_predictions)
LR_F1_Score = metrics.f1_score(y_test, LR_predictions)
LR_Log_Loss = metrics.log_loss(y_test, LR_predict_proba)

### SVM

In [21]:
#Train the Support Vector Model Model
SVM.fit(x_train, y_train)

#Predict trained model using data test
SVM_predictions = SVM.predict(x_test)

In [22]:
#Calculate the Evaluation Metrics 
SVM_Accuracy_Score = metrics.accuracy_score(y_test,SVM_predictions)
SVM_JaccardIndex = metrics.jaccard_score(y_test,SVM_predictions)
SVM_F1_Score = metrics.f1_score(y_test,SVM_predictions)

### Report 
save all the metrics into dataframe and show it 

In [23]:
Report = pd.DataFrame({
    'Metrics': ['Accuracy', 'Jaccard Index', 'F1-Score', 'LogLoss', 'MAE', 'MSE', 'R2'],
    'Linear Regression': ['-', '-', '-', '-',LinearRegression_MAE,LinearRegression_MSE,LinearRegression_R2],
    'KNN': [KNN_Accuracy_Score, KNN_JaccardIndex, KNN_F1_Score,'-', '-', '-', '-', ],
    'Decision Tree':[Tree_Accuracy_Score, Tree_JaccardIndex, Tree_F1_Score, '-', '-', '-', '-'],
    'Logistic Regression':[LR_Accuracy_Score, LR_JaccardIndex, LR_F1_Score, LR_Log_Loss, '-', '-', '-'],
    'SVM':[SVM_Accuracy_Score, SVM_JaccardIndex, SVM_F1_Score, '-', '-', '-', '-']
})
Report.head(7)

,Metrics,Linear Regression,KNN,Decision Tree,Logistic Regression,SVM
0,Accuracy,-,0.818321,0.819847,0.841221,0.719084
1,Jaccard Index,-,0.425121,0.470852,0.527273,0.0
2,F1-Score,-,0.59661,0.640244,0.690476,0.0
3,LogLoss,-,-,-,0.357201,-
4,MAE,0.256319,-,-,-,-
5,MSE,0.115722,-,-,-,-
6,R2,0.427126,-,-,-,-
